In [1]:
InputDate = "2021-12-28"

In [2]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime, timedelta
ReportDate = pd.to_datetime(InputDate, format='%Y-%m-%d').strftime("%Y/%m/%d")
ReportDate = datetime.strptime(ReportDate, "%Y/%m/%d")

In [4]:
import os
Fileurl = os.getcwd()
datefile = Fileurl + "/" + InputDate
if not os.path.exists(datefile):
    os.makedirs(datefile)

In [5]:
if ReportDate.isoweekday() == 1:
    LastReportDay = 3
else:
    LastReportDay = 1
LastDate = ReportDate - timedelta(LastReportDay)
LastDate = pd.to_datetime(LastDate, format='%Y-%m-%d').strftime("%Y/%m/%d")
ReportDate = pd.to_datetime(ReportDate, format='%Y-%m-%d').strftime("%Y/%m/%d")

In [6]:
today = pd.to_datetime(pd.to_datetime("today"), format='%Y-%m-%d').strftime("%Y%m%d")
yahoo_start = InputDate
start = pd.to_datetime(yahoo_start, format='%Y-%m-%d').strftime("%Y/%m/%d")
start  = datetime.strptime(start , "%Y/%m/%d")
yahoo_end = start + timedelta(1)
yahoo_end = pd.to_datetime(yahoo_end, format='%Y-%m-%d').strftime("%Y-%m-%d")

In [7]:
Excel = pd.ExcelFile(Fileurl + "/CCASS List.xlsx")
CCASS_list = pd.read_excel(Excel, index_col = 0)
CCASS_list = CCASS_list.drop(['Name'], axis=1)
CCASS_list.index = CCASS_list.index.astype(str)
CCASS_list

,Yahoo
HKEX,
1,0001.HK
2,0002.HK
3,0003.HK
4,0004.HK
5,0005.HK
...,...
77973,300973.SZ
77979,300979.SZ
77981,300981.SZ


In [8]:
import urllib
import requests
from bs4 import BeautifulSoup

def Get_Report(LastDate, ReportDate, today, CCASS_list, Ticker):
    Date = LastDate
    url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=' + Ticker
    
    post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
    response = requests.post(url, data=post_fields)
    content = response.content
    soup = BeautifulSoup(content)
    body = soup.find_all("div",{"class":"mobile-list-body"})
    SZ = []
    for i in range(int(len(body)/4)):
        ID = body[i*4].text
        name = body[i*4+1].text
        share = body[i*4+2].text
        percent = body[i*4+3].text
        SZ.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})
        SZ_df = pd.DataFrame(SZ)
        SZ_df = SZ_df.set_index('ID')
        SZ_df['Share']=SZ_df['Share'].str.replace(',','').astype(float)
        SZ_df.index = SZ_df.index.str.strip()
    
    
    Date = ReportDate
    url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=' + Ticker 
    post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
    response = requests.post(url, data=post_fields)
    content = response.content
    soup = BeautifulSoup(content)
    body = soup.find_all("div",{"class":"mobile-list-body"})
    now_SZ = []
    for i in range(int(len(body)/4)):
        ID = body[i*4].text
        name = body[i*4+1].text
        share = body[i*4+2].text
        percent = body[i*4+3].text
        now_SZ.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})
    
    now_SZ_df = pd.DataFrame(now_SZ)
    now_SZ_df = now_SZ_df.set_index('ID')
    now_SZ_df['Share']=now_SZ_df['Share'].str.replace(',','').astype(float)
    now_SZ_df.index = now_SZ_df.index.str.strip()
    now_SZ_df["Share Change"] = now_SZ_df["Share"] - SZ_df["Share"]
    
    com_SZ = pd.concat([now_SZ_df, CCASS_list], axis=1, join="inner")
    #com_SZ = com_SZ[com_SZ["Share Change"]!=0]
    
    return com_SZ    

In [9]:
com_SZ = Get_Report(LastDate, ReportDate, today, CCASS_list, 'sz')
com_SZ = com_SZ[com_SZ["Share Change"]!=0]
com_SH = Get_Report(LastDate, ReportDate, today, CCASS_list, 'sh')
com_SH = com_SH[com_SH["Share Change"]!=0]
com_HK = Get_Report(LastDate, ReportDate, today, CCASS_list, 'hk')
com_HK = com_HK[com_HK["Share Change"]!=0]

In [14]:
import yfinance as yf
com_SZ["price"] = ''
for a in range(len(com_SZ)):
    com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]
<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-14-f110794cbde9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SZ["price"][a] = yf.download(tickers = com_SZ["Yahoo"][a], start=yahoo_start, end=yahoo_end)["Close"][-1]


In [16]:
com_SZ["money flow"] = com_SZ["Share Change"] * com_SZ["price"]
com_SZ = com_SZ.drop(['Percent'], axis=1)
com_SZ = com_SZ.set_index('Yahoo')
com_SZ

KeyError: "['Percent'] not found in axis"

In [17]:
com_SH["price"] = ''
for b in range(len(com_SH)):
    com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]
<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]
<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-17-b8398b9cf83d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_SH["price"][b] = yf.download(tickers = com_SH["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


In [18]:
com_SH["money flow"] = com_SH["Share Change"] * com_SH["price"]
com_SH = com_SH.drop(['Percent'], axis=1)
com_SH = com_SH.set_index('Yahoo')
com_SH

,Name,Share,Share Change,price,money flow
Yahoo,,,,,
688002.SS,"RAYTRON TECHNOLOGY CO.,LTD. (A #688002)",6491068.0,54531.0,77.290001,4214701.039925
688005.SS,"NINGBO RONBAY NEW ENERGY TECHNOLOGY CO., LTD. ...",1465077.0,193457.0,109.0,21086813.0
688006.SS,ZHEJIANG HANGKE TECHNOLOGY INCORPORATED COMPAN...,14997925.0,6851.0,105.730003,724356.252998
688008.SS,"MONTAGE TECHNOLOGY CO., LTD. (A #688008)",28032206.0,145706.0,79.849998,11634623.87767
688009.SS,CHINA RAILWAY SIGNAL & COMMUNICATION CORPORATI...,81912695.0,-114206.0,4.98,-568745.882178
...,...,...,...,...,...
605296.SS,YUNNAN SHENNONG AGRICULTURAL INDUSTRY GROUP CO...,103600.0,42600.0,40.450001,1723170.032501
605337.SS,"ZHE JIANG LI ZI YUAN FOOD CO,LTD (A #605337)",566538.0,-137318.0,49.209999,-6757418.654282
605358.SS,"HANGZHOU LION MICROELECTRONICS CO.,LTD. (A #60...",596400.0,88700.0,124.25,11020975.0


In [19]:
com_HK["price"] = ''
for b in range(len(com_HK)):
    com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]

[*********************100%***********************]  1 of 1 completed


<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


...

<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]
<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]
<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


[*********************100%***********************]  1 of 1 completed


<ipython-input-19-0e984e53b213>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_HK["price"][b] = yf.download(tickers = com_HK["Yahoo"][b], start=yahoo_start, end=yahoo_end)["Close"][-1]


In [20]:
com_HK["money flow"] = com_HK["Share Change"] * com_HK["price"]
com_HK = com_HK.drop(['Percent'], axis=1)
com_HK = com_HK.set_index('Yahoo')
com_HK

,Name,Share,Share Change,price,money flow
Yahoo,,,,,
0001.HK,CK HUTCHISON HOLDINGS LIMITED,3.323890e+07,230000.0,51.0,11730000.0
0002.HK,CLP HOLDINGS LIMITED,4.089992e+06,-7000.0,79.349998,-555449.989319
0003.HK,"HONG KONG AND CHINA GAS COMPANY LIMITED, THE",3.964281e+07,594000.0,12.16,7223039.909363
0004.HK,"WHARF (HOLDINGS) LIMITED, THE",4.410450e+06,-2000.0,23.65,-47299.999237
0005.HK,HSBC HOLDINGS PLC,1.926011e+09,1124400.0,46.599998,52397038.284302
...,...,...,...,...,...
9992.HK,POP MART INTERNATIONAL GROUP LIMITED,1.918679e+08,-404200.0,44.200001,-17865640.30838
9993.HK,RADIANCE HOLDINGS (GROUP) COMPANY LIMITED,1.009240e+08,-1000.0,4.05,-4050.000191
9995.HK,"REMEGEN CO., LTD.",1.948250e+07,401000.0,71.849998,28811849.388123


In [21]:
writer = pd.ExcelWriter(Fileurl + "/" + InputDate + "/" + InputDate + " Summary.xlsx", engine='xlsxwriter')
com_SZ.to_excel(writer, sheet_name='Northbound SZ')
com_SH.to_excel(writer, sheet_name='Northbound SH')
com_HK.to_excel(writer, sheet_name='Southbound HK')
writer.save()

In [22]:
def new_report(LastDate, ReportDate, today, CCASS_list, ticker):
    temp_SZ = Get_Report(LastDate, ReportDate, today, CCASS_list, ticker)
    temp_SZ = temp_SZ.set_index('Yahoo')
    if ticker == "sz":
        temp_SZ = pd.concat([temp_SZ, com_SZ['price']], axis=1, join="inner")
    elif ticker == 'sh':
        temp_SZ = pd.concat([temp_SZ, com_SH['price']], axis=1, join="inner")
    elif ticker == 'hk':
        temp_SZ = pd.concat([temp_SZ, com_HK['price']], axis=1, join="inner")            
            
    temp_SZ["money flow"] = temp_SZ["Share Change"] * temp_SZ["price"]
    temp_SZ = temp_SZ.drop(['Percent'], axis=1)
    return temp_SZ

In [23]:
com_SZ

,Name,Share,Share Change,price,money flow
Yahoo,,,,,
000001.SZ,"PING AN BANK CO., LTD. (A #000001)",1.754374e+09,-20146763.0,17.17,-345919922.247076
000002.SZ,"CHINA VANKE CO., LTD. (A #000002)",3.349826e+08,160058.0,19.700001,3153142.722115
000006.SZ,"SHENZHEN ZHENYE (GROUP) CO.,LTD. (A #000006)",6.570427e+06,-141301.0,4.37,-617485.353829
000008.SZ,"CHINA HIGH-SPEED RAILWAY TECHNOLOGY CO., LTD. ...",4.007265e+07,2410653.0,2.73,6581082.73598
000009.SZ,"CHINA BAOAN GROUP CO.,LTD. (A #000009)",7.283850e+07,7971899.0,14.24,113519839.935377
...,...,...,...,...,...
300973.SZ,LIGAO FOODS CO.LTD (A #300973),1.032310e+05,14900.0,137.009995,2041448.918152
300979.SZ,HUALI INDUSTRIAL GROUP COMPANY LIMITED (A #300...,3.683340e+05,67575.0,84.57,5714817.729378
300981.SZ,"ZHONGHONG PULIN MEDICAL PRODUCTS CO., LTD. (A ...",1.462960e+05,-4748.0,78.449997,-372480.58551


In [24]:
SZ_report = com_SZ.copy()
SH_report = com_SH.copy()
HK_report = com_HK.copy()
for i in range(1,8):
    time = pd.to_datetime(InputDate, format='%Y-%m-%d').strftime("%Y/%m/%d")
    time  = datetime.strptime(time , "%Y/%m/%d")
    datadate = time - timedelta(i)
    if datadate.isoweekday() < 6:
        datadate = pd.to_datetime(datadate, format='%Y-%m-%d').strftime("%Y-%m-%d") 
        datefile = Fileurl + "/" + datadate
        if not os.path.exists(datefile):
            os.makedirs(datefile)
            ReportDate = pd.to_datetime(datadate, format='%Y-%m-%d').strftime("%Y/%m/%d")
            ReportDate = datetime.strptime(ReportDate, "%Y/%m/%d")  
            LastDate = ReportDate - timedelta(1)
            LastDate = pd.to_datetime(LastDate, format='%Y-%m-%d').strftime("%Y/%m/%d")
            ReportDate = pd.to_datetime(ReportDate, format='%Y-%m-%d').strftime("%Y/%m/%d")
            
            temp_SZ = new_report(LastDate, ReportDate, today, CCASS_list, 'sz')
            temp_SH = new_report(LastDate, ReportDate, today, CCASS_list, 'sh')
            temp_HK = new_report(LastDate, ReportDate, today, CCASS_list, 'hk')
            
            writer = pd.ExcelWriter(Fileurl + "/" + datadate  + "/" + datadate  + " Summary.xlsx", engine='xlsxwriter')
            temp_SZ.to_excel(writer, sheet_name='Northbound SZ')
            temp_SH.to_excel(writer, sheet_name='Northbound SH')
            temp_HK.to_excel(writer, sheet_name='Southbound HK')
            writer.save()
             
        
        excelfile = pd.ExcelFile(Fileurl + "/" + datadate + "/" + datadate + " Summary.xlsx")
        last_SZ = pd.read_excel(excelfile, "Northbound SZ",index_col = 0)
        last_SZ = last_SZ[["Share Change", "money flow"]]
        last_SZ = last_SZ.rename(columns={'Share Change': datadate + ' Share Change', 'money flow': datadate + ' money flow'})
        SZ_report = pd.concat([SZ_report, last_SZ], axis=1, join="inner")
            
        last_SH = pd.read_excel(excelfile, "Northbound SH",index_col = 0)
        last_SH = last_SH[["Share Change", "money flow"]]
        last_SH = last_SH.rename(columns={'Share Change': datadate + ' Share Change', 'money flow': datadate + ' money flow'})
        SH_report = pd.concat([SH_report, last_SH], axis=1, join="inner")
            
        last_HK = pd.read_excel(excelfile, "Southbound HK",index_col = 0)
        last_HK = last_HK[["Share Change", "money flow"]]
        last_HK = last_HK.rename(columns={'Share Change': datadate + ' Share Change', 'money flow': datadate + ' money flow'})
        HK_report = pd.concat([HK_report, last_HK], axis=1, join="inner")

In [25]:
SZ_report

,Name,Share,Share Change,price,money flow,2021-12-27 Share Change,2021-12-27 money flow,2021-12-24 Share Change,2021-12-24 money flow,2021-12-23 Share Change,2021-12-23 money flow,2021-12-22 Share Change,2021-12-22 money flow,2021-12-21 Share Change,2021-12-21 money flow
Yahoo,,,,,,,,,,,,,,,
000001.SZ,"PING AN BANK CO., LTD. (A #000001)",1.754374e+09,-20146763.0,17.17,-345919922.247076,0,0,0,0,-4082643,-7.067055e+07,-21440944,-3.728580e+08,-20302646,-3.575296e+08
000002.SZ,"CHINA VANKE CO., LTD. (A #000002)",3.349826e+08,160058.0,19.700001,3153142.722115,0,0,0,0,-4065414,-7.984473e+07,-3008274,-5.974432e+07,8465032,1.718401e+08
000006.SZ,"SHENZHEN ZHENYE (GROUP) CO.,LTD. (A #000006)",6.570427e+06,-141301.0,4.37,-617485.353829,0,0,0,0,608628,2.696222e+06,-284652,-1.283781e+06,162727,7.680714e+05
000008.SZ,"CHINA HIGH-SPEED RAILWAY TECHNOLOGY CO., LTD. ...",4.007265e+07,2410653.0,2.73,6581082.73598,0,0,0,0,-1925516,-6.315692e+06,1254500,3.738410e+06,-2103300,-5.699943e+06
000009.SZ,"CHINA BAOAN GROUP CO.,LTD. (A #000009)",7.283850e+07,7971899.0,14.24,113519839.935377,0,0,0,0,358002,5.319910e+06,600404,9.066101e+06,-958831,-1.443041e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300973.SZ,LIGAO FOODS CO.LTD (A #300973),1.032310e+05,14900.0,137.009995,2041448.918152,0,0,0,0,6600,9.249240e+05,17561,2.511223e+06,-9951,-1.394732e+06
300979.SZ,HUALI INDUSTRIAL GROUP COMPANY LIMITED (A #300...,3.683340e+05,67575.0,84.57,5714817.729378,0,0,0,0,-21364,-1.881741e+06,-1680,-1.478568e+05,12311,1.066502e+06
300981.SZ,"ZHONGHONG PULIN MEDICAL PRODUCTS CO., LTD. (A ...",1.462960e+05,-4748.0,78.449997,-372480.58551,0,0,0,0,19883,1.562605e+06,53600,4.301936e+06,20006,1.588677e+06


In [26]:
SZ_report = SZ_report.rename(columns={'Share': InputDate + ' Share', 'Share Change': InputDate + ' Share Change', 'money flow': InputDate + ' money flow'})
SZ_report = SZ_report.iloc[:,[0,1,3,2,5,7,9,11,13,4,6,8,10,12,14]]

SH_report = SH_report.rename(columns={'Share': InputDate + ' Share', 'Share Change': InputDate + ' Share Change', 'money flow': InputDate + ' money flow'})
SH_report = SH_report.iloc[:,[0,1,3,2,5,7,9,11,13,4,6,8,10,12,14]]

HK_report = HK_report.rename(columns={'Share': InputDate + ' Share', 'Share Change': InputDate + ' Share Change', 'money flow': InputDate + ' money flow'})
HK_report = HK_report.iloc[:,[0,1,3,2,5,7,9,11,13,4,6,8,10,12,14]]

In [27]:
SZ_report["Signal"] = ''
SH_report["Signal"] = ''
HK_report["Signal"] = ''

for a in SZ_report.index:
    if SZ_report.iloc[:,3][a] >= 0 and SZ_report.iloc[:,4][a] >= 0 and SZ_report.iloc[:,5][a] >= 0 and SZ_report.iloc[:,6][a] >= 0 and SZ_report.iloc[:,7][a] >= 0: 
        SZ_report["Signal"][a] = "Buy Trades"
    elif SZ_report.iloc[:,3][a] <= 0 and SZ_report.iloc[:,4][a] <= 0 and SZ_report.iloc[:,5][a] <= 0 and SZ_report.iloc[:,6][a] <= 0 and SZ_report.iloc[:,7][a] <= 0: 
        SZ_report["Signal"][a] = "Sell Trades"
    else:
        SZ_report["Signal"][a] = "No Pattern"
        
for b in SH_report.index:
    if SH_report.iloc[:,3][b] >= 0 and SH_report.iloc[:,4][b] >= 0 and SH_report.iloc[:,5][b] >= 0 and SH_report.iloc[:,6][b] >= 0 and SH_report.iloc[:,7][b] >= 0: 
        SH_report["Signal"][b] = "Buy Trades"
    elif SH_report.iloc[:,3][b] <= 0 and SH_report.iloc[:,4][b] <= 0 and SH_report.iloc[:,5][b] <= 0 and SH_report.iloc[:,6][b] <= 0 and SH_report.iloc[:,7][b] <= 0: 
        SH_report["Signal"][b] = "Sell Trades"
    else:
        SH_report["Signal"][b] = "No Pattern"

for c in HK_report.index:
    if HK_report.iloc[:,3][c] >= 0 and HK_report.iloc[:,4][c] >= 0 and HK_report.iloc[:,5][c] >= 0 and HK_report.iloc[:,6][c] >= 0 and HK_report.iloc[:,7][c] >= 0: 
        HK_report["Signal"][c] = "Buy Trades"
    elif HK_report.iloc[:,3][c] <= 0 and HK_report.iloc[:,4][c] <= 0 and HK_report.iloc[:,5][c] <= 0 and HK_report.iloc[:,6][c] <= 0 and HK_report.iloc[:,7][c] <= 0: 
        HK_report["Signal"][c] = "Sell Trades"
    else:
        HK_report["Signal"][c] = "No Pattern"

<ipython-input-27-a6c6567535ea>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SZ_report["Signal"][a] = "Sell Trades"
<ipython-input-27-a6c6567535ea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SZ_report["Signal"][a] = "No Pattern"
<ipython-input-27-a6c6567535ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SZ_report["Signal"][a] = "Buy Trades"
<ipython-input-27-a6c6567535ea>:19: SettingWithCopyWarning: 
A value is trying t

In [28]:
Rwriter = pd.ExcelWriter(Fileurl + "/" + InputDate + "/" + InputDate + " Report.xlsx", engine='xlsxwriter')
SZ_report.to_excel(Rwriter, sheet_name='Northbound SZ')
SH_report.to_excel(Rwriter, sheet_name='Northbound SH')
HK_report.to_excel(Rwriter, sheet_name='Southbound HK')
Rwriter.save()

In [ ]:
'''
Date = LastDate
url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=hk' 
post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
response = requests.post(url, data=post_fields)
content = response.content
soup = BeautifulSoup(content)
body = soup.find_all("div",{"class":"mobile-list-body"})
for i in range(int(len(body)/4)):
    ID = body[i*4].text
    name = body[i*4+1].text
    share = body[i*4+2].text
    percent = body[i*4+3].text
    HK.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})
    
HK_df = pd.DataFrame(HK)
HK_df = HK_df.set_index('ID')
HK_df['Share']=HK_df['Share'].str.replace(',','').astype(float)
HK_df.index = HK_df.index.str.strip()

Date = ReportDate
url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=hk' 
post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
response = requests.post(url, data=post_fields)
content = response.content
soup = BeautifulSoup(content)
body = soup.find_all("div",{"class":"mobile-list-body"})
now_HK = []
for i in range(int(len(body)/4)):
    ID = body[i*4].text
    name = body[i*4+1].text
    share = body[i*4+2].text
    percent = body[i*4+3].text
    now_HK.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})
    
now_HK_df = pd.DataFrame(now_HK)
now_HK_df = now_HK_df.set_index('ID')
now_HK_df['Share']=now_HK_df['Share'].str.replace(',','').astype(float)
now_HK_df.index = now_HK_df.index.str.strip()
now_HK_df["Share Change"] = now_HK_df["Share"] - HK_df["Share"]

com_HK = pd.concat([now_HK_df, CCASS_list], axis=1, join="inner")
com_HK = com_HK[com_HK["Share Change"]!=0]
'''

In [ ]:
'''
Date = LastDate
url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=sh' 
post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
response = requests.post(url, data=post_fields)
content = response.content
soup = BeautifulSoup(content)
body = soup.find_all("div",{"class":"mobile-list-body"})
SH = []
for i in range(int(len(body)/4)):
    ID = body[i*4].text
    name = body[i*4+1].text
    share = body[i*4+2].text
    percent = body[i*4+3].text
    SH.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})
    
SH_df = pd.DataFrame(SH)
SH_df = SH_df.set_index('ID')
SH_df['Share']=SH_df['Share'].str.replace(',','').astype(float)

Date = ReportDate
url = 'https://www.hkexnews.hk/sdw/search/mutualmarket.aspx?t=sh' 
post_fields = {'MIME Type':'application/x-www-form-urlencoded','__VIEWSTATE':'/wEPDwUJNjIxMTYzMDAwZGSFj8kdzCLeVLiJkFRvN5rjsPotqw==','__VIEWSTATEGENERATOR':'3C67932C','__EVENTVALIDATION': '/wEdAAdbi0fj+ZSDYaSP61MAVoEdVobCVrNyCM2j+bEk3ygqmn1KZjrCXCJtWs9HrcHg6Q64ro36uTSn/Z2SUlkm9HsG7WOv0RDD9teZWjlyl84iRMtpPncyBi1FXkZsaSW6dwqO1N1XNFmfsMXJasjxX85ju3P1WAPUeweM/r0/uwwyYLgN1B8=','today':today,'sortBy':'stockcode','sortDirection':'asc','txtShareholdingDate':Date,'btnSearch':'Search'}     # Set POST fields here
response = requests.post(url, data=post_fields)
content = response.content
soup = BeautifulSoup(content)
body = soup.find_all("div",{"class":"mobile-list-body"})
now_SH = []
for i in range(int(len(body)/4)):
    ID = body[i*4].text
    name = body[i*4+1].text
    share = body[i*4+2].text
    percent = body[i*4+3].text
    now_SH.append({"ID":ID, "Name":name, "Share":share, "Percent":percent})

now_SH_df = pd.DataFrame(now_SH)
now_SH_df = now_SH_df.set_index('ID')
now_SH_df['Share']=now_SH_df['Share'].str.replace(',','').astype(float)
now_SH_df["Share Change"] = now_SH_df["Share"] - SH_df["Share"]

com_SH = pd.concat([now_SH_df, CCASS_list], axis=1, join="inner")
com_SH = com_SH[com_SH["Share Change"]!=0]

'''

In [ ]:
'''   
    for a in SZ_report.index:
            if SZ_report["Signal"][a] == "YES" and SZ_report[datadate + ' Share Change'][a] > 0:
                SZ_report["Signal"][a] = "YES"
            else:
                SZ_report["Signal"][a] = "NO"
        
        for b in SH_report.index:
            if SH_report["Signal"][b] == "YES" and SH_report[datadate + ' Share Change'][b] > 0:
                SH_report["Signal"][b] = "YES"
            else:
                SH_report["Signal"][b] = "NO"  
                
        for c in HK_report.index:
            if HK_report["Signal"][c] == "YES" and HK_report[datadate + ' Share Change'][c] > 0:
                HK_report["Signal"][c] = "YES"
            else:
                HK_report["Signal"][c] = "NO" 
''' 

In [ ]:
'''
import urllib
import requests
from bs4 import BeautifulSoup
sublist = []
for i in range(len(CCASS)):
    StockCode = CCASS[i]
    Date = LastDate
    url = 'https://www.hkexnews.hk/sdw/search/searchsdw.aspx' 
    post_fields = {'MIME Type':'application/x-www-form-urlencoded','__EVENTTARGET':'btnSearch','__VIEWSTATE':'/wEPDwULLTIwNTMyMzMwMThkZLiCLeQCG/lBVJcNezUV/J0rsyMr','__VIEWSTATEGENERATOR': 'A7B2BBE2','today':today,'sortBy':'shareholding','sortDirection':'desc','txtShareholdingDate':Date,'txtStockCode':StockCode}     # Set POST fields here
    response = requests.post(url, data=post_fields)
    content = response.content
    soup = BeautifulSoup(content)
    share = soup.find_all("div",{"class":"ccass-search-datarow"})[0].find_all("div",{"class":"value"})[0].text
    share = float(share.replace(',',''))
    sublist.append(share)
'''

In [ ]:
'''
next_sublist = []
for j in range(len(CCASS)):
    StockCode = CCASS[j]
    Date = ReportDate
    url = 'https://www.hkexnews.hk/sdw/search/searchsdw.aspx' # Set destination URL here
    post_fields = {'MIME Type':'application/x-www-form-urlencoded','__EVENTTARGET':'btnSearch','__VIEWSTATE':'/wEPDwULLTIwNTMyMzMwMThkZLiCLeQCG/lBVJcNezUV/J0rsyMr','__VIEWSTATEGENERATOR': 'A7B2BBE2','today':today,'sortBy':'shareholding','sortDirection':'desc','txtShareholdingDate':Date,'txtStockCode':StockCode}     # Set POST fields here
    next_response = requests.post(url, data=post_fields)
    next_content = next_response.content
    next_soup = BeautifulSoup(next_content)
    next_share = next_soup.find_all("div",{"class":"ccass-search-datarow"})[0].find_all("div",{"class":"value"})[0].text
    next_share = float(next_share.replace(',',''))
    next_sublist.append(next_share)
'''

In [ ]:
'''
import yfinance as yf
diff_list = []
for k in range(len(CCASS)):
    price =  yf.download(tickers = Yahoo[k], start="2021-12-20", end="2021-12-21")["Close"][-1]
    diff = (next_sublist[k] - sublist[k]) * price
    print(price)
    diff_list.append(diff)
'''

In [ ]:
#CCASS = ["90036","90519","90809","91012","91318","91669","91888","91919","93259","93501"]
#Yahoo = ["600036.SS","600519.SS","600809.SS","601012.SS","601318.SS","601669.SS","601888.SS","601919.SS","603259.SS","603501.SS"]

#CCASS = ["77750","77059","72241","72594","77014","72475","70858","77274","72812","72460"]
#Yahoo = ["300750.SZ","300059.SZ","002241.SZ","002594.SZ","300014.SZ","002475.SZ","000858.SZ","300274.SZ","002812.SZ","002460.SZ"]

#CCASS = ["00700","03690","01810","02269","01918","00836","02020","00981","02331","02382"]
#Yahoo = ["0700.HK","3690.HK","1810.HK","2269.HK","1918.HK","0836.HK","2020.HK","0981.HK","2331.HK","2382.HK"]


In [ ]:
'''
total = len(soup.find_all("td",{"class":"col-participant-id"}))
sublist = []
for i in range(total):
    ID = soup.find_all("td",{"class":"col-participant-id"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    Name = soup.find_all("td",{"class":"col-participant-name"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    share = soup.find_all("td",{"class":"col-shareholding text-right"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    percent = soup.find_all("td",{"class":"col-shareholding-percent text-right"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    sublist.append({"ID":ID, "Name":Name, "Share":share, "Percent":percent})
'''

In [ ]:
'''
df = pd.DataFrame(sublist)
df['Share']=df['Share'].str.replace(',','').astype(float)
df["ID"] = df.apply(lambda x: x["Name"] if x['ID'] ==''  else x['ID'], axis=1)
df = df.set_index('ID')
df
'''

In [ ]:
'''
StockCode = "00939"
today = today
Date = ReportDate
url = 'https://www.hkexnews.hk/sdw/search/searchsdw.aspx' # Set destination URL here
post_fields = {'MIME Type':'application/x-www-form-urlencoded','__EVENTTARGET':'btnSearch','__VIEWSTATE':'/wEPDwULLTIwNTMyMzMwMThkZLiCLeQCG/lBVJcNezUV/J0rsyMr','__VIEWSTATEGENERATOR': 'A7B2BBE2','today':today,'sortBy':'shareholding','sortDirection':'desc','txtShareholdingDate':Date,'txtStockCode':StockCode}     # Set POST fields here
next_response = requests.post(url, data=post_fields)
next_content = next_response.content
next_soup = BeautifulSoup(next_content)
'''

In [ ]:
#next_share = next_soup.find_all("div",{"class":"ccass-search-datarow"})[0].find_all("div",{"class":"value"})[0].text

In [ ]:
'''
next_total = len(next_soup.find_all("td",{"class":"col-participant-id"}))
next_sublist = []
for i in range(next_total):
    next_ID = next_soup.find_all("td",{"class":"col-participant-id"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    next_Name = next_soup.find_all("td",{"class":"col-participant-name"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    next_share = next_soup.find_all("td",{"class":"col-shareholding text-right"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    next_percent = next_soup.find_all("td",{"class":"col-shareholding-percent text-right"})[i].find_all("div",{"class":"mobile-list-body"})[0].text
    next_sublist.append({"Next ID":next_ID , "Next Name":next_Name, "Next Share":next_share, "Next Percent":next_percent})
'''

In [ ]:
'''
next_df = pd.DataFrame(next_sublist)
next_df['Next Share']=next_df['Next Share'].str.replace(',','').astype(float)
next_df["Next ID"] = next_df.apply(lambda x: x["Next Name"] if x['Next ID'] == '' else x['Next ID'], axis=1)
next_df = next_df.set_index('Next ID')
next_df
'''

In [ ]:
'''
total_df = pd.concat([df, next_df], axis=1)
total_df['Share'] = total_df['Share'].fillna(0)
total_df['Next Share'] = total_df['Next Share'].fillna(0)
total_df["Diff"] = total_df["Next Share"] - total_df["Share"] 
total_df
'''

In [ ]:
#total_df["Diff"].sum() * 5.27

In [ ]:
#next_df["sell"] = next_df.apply(lambda x: x["diff"] if x['diff'] > 0  else 0, axis=1)
#next_df["buy"] = next_df.apply(lambda x: -x["diff"] if x['diff'] < 0  else 0, axis=1)